In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
import psycopg2
from psycopg2 import OperationalError
import os
from dotenv import load_dotenv

In [3]:
# Definindo parâmetros para a geração de dados
np.random.seed(0)  # Para garantir a reprodutibilidade
tipos_cerveja = ["Pilsen", "IPA", "Stout", "Lager", "Ale", "Wheat Beer", "Porter"]
quantidade_min, quantidade_max = 500, 5000  # Quantidade produzida por lote

In [4]:
# Gerando dados de produção (Lotes)
dados_producao = pd.DataFrame({
    "ID_Lote": range(1, 251),
    "Tipo_Cerveja": np.random.choice(tipos_cerveja, 250),
    "Data_Producao": pd.date_range(start="2020-01-01", periods=250, freq='D'),
    "Quantidade_Produzida": np.random.randint(quantidade_min, quantidade_max, 250)
})

In [5]:
# Dados de vendas (2000 entradas ao longo de 3 anos)
datas_vendas = pd.date_range(start="2020-01-01", end="2022-12-31", freq='D')
vendas_por_data = np.random.choice(datas_vendas, 2000)
dados_vendas = pd.DataFrame({
    "ID_Venda": range(1, 2001),
    "ID_Lote": np.random.choice(dados_producao["ID_Lote"], 2000),
    "Data_Venda": vendas_por_data,
    "Quantidade_Vendida": np.random.randint(1, 100, 2000),  # Quantidade vendida por transação
    "Preco_Venda": np.random.uniform(5, 50, 2000)  # Preço aleatório entre 5 e 50
})

In [6]:
# Dados de devolução (13% das vendas)
quantidade_devolucoes = int(0.13 * len(dados_vendas))
dados_devolucao = dados_vendas.sample(n=quantidade_devolucoes).copy()
dados_devolucao.rename(columns={"Quantidade_Vendida": "Quantidade_Devolvida"}, inplace=True)
dados_devolucao["ID_Devolucao"] = range(1, quantidade_devolucoes + 1)
dados_devolucao["Motivo_Devolucao"] = np.random.choice(["Defeito", "Insatisfação", "Outro"], quantidade_devolucoes)


In [7]:
# Dados de feedback (gerados aleatoriamente)
quantidade_feedbacks = random.randint(100, 300)  # Quantidade aleatória de feedbacks
dados_feedback = pd.DataFrame({
    "ID_Feedback": range(1, quantidade_feedbacks + 1),
    "ID_Venda": np.random.choice(dados_vendas["ID_Venda"], quantidade_feedbacks),
    "Data_Feedback": np.random.choice(datas_vendas, quantidade_feedbacks),
    "Classificacao": np.random.randint(1, 6, quantidade_feedbacks),  # Classificação de 1 a 5
    "Comentarios": np.random.choice(["Muito bom", "Bom", "Regular", "Ruim", "Muito ruim"], quantidade_feedbacks)
})

dados_producao.head(), dados_vendas.head(), dados_devolucao.head(), dados_feedback.head()

(   ID_Lote Tipo_Cerveja Data_Producao  Quantidade_Produzida
 0        1          Ale    2020-01-01                  4887
 1        2   Wheat Beer    2020-01-02                  4281
 2        3       Pilsen    2020-01-03                  3275
 3        4        Lager    2020-01-04                  3103
 4        5        Lager    2020-01-05                  1662,
    ID_Venda  ID_Lote Data_Venda  Quantidade_Vendida  Preco_Venda
 0         1      244 2022-09-25                  11    23.210412
 1         2       94 2022-03-26                   5    19.827663
 2         3       59 2020-04-14                  74    16.968799
 3         4      112 2021-10-02                  91    19.087046
 4         5      154 2020-11-23                   9    46.587291,
       ID_Venda  ID_Lote Data_Venda  Quantidade_Devolvida  Preco_Venda  \
 1375      1376      121 2022-06-25                    85    46.704618   
 1482      1483      176 2022-12-24                    52    33.156435   
 1632      163

In [8]:
# Carrega as variáveis de ambiente do arquivo .env
load_dotenv()

# Obtém as informações de conexão do arquivo .env
host = os.getenv('DB_HOST', 'localhost')
porta = os.getenv('DB_PORT', '5432')
dbname = os.getenv('DB_NAME', 'postgres')
usuario = os.getenv('DB_USER', 'postgres')
senha = os.getenv('DB_PASSWORD', 'default_password')

def create_connection(host_name, port, database_name, user_name, user_password):
    connection = None
    try:
        connection = psycopg2.connect(
            host=host_name,
            port=port,
            dbname=database_name,
            user=user_name,
            password=user_password
        )
        print("Conexão com o PostgreSQL estabelecida")
    except OperationalError as e:
        print(f"Ocorreu um erro ao conectar ao PostgreSQL: {e}")
    return connection

# Estabelecendo a conexão
conn = create_connection(host, porta, dbname, usuario, senha)

# Lembre-se de fechar a conexão quando terminar
# conn.close()

Conexão com o PostgreSQL estabelecida


In [9]:
def create_tables(connection):
    cursor = connection.cursor()

    # Criação da tabela de produção
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Producao (
        ID_Lote INTEGER PRIMARY KEY,
        Tipo_Cerveja TEXT,
        Data_Producao DATE,
        Quantidade_Produzida INTEGER
    )
    """)

    # Criação da tabela de vendas
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Vendas (
        ID_Venda INTEGER PRIMARY KEY,
        ID_Lote INTEGER,
        Data_Venda DATE,
        Quantidade_Vendida INTEGER,
        Preco_Venda REAL,
        FOREIGN KEY (ID_Lote) REFERENCES Producao (ID_Lote)
    )
    """)

    # Criação da tabela de devoluções
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Devolucoes (
        ID_Devolucao INTEGER PRIMARY KEY,
        ID_Venda INTEGER,
        Quantidade_Devolvida INTEGER,
        Motivo_Devolucao TEXT,
        FOREIGN KEY (ID_Venda) REFERENCES Vendas (ID_Venda)
    )
    """)

    # Criação da tabela de feedback
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Feedback (
        ID_Feedback INTEGER PRIMARY KEY,
        ID_Venda INTEGER,
        Data_Feedback DATE,
        Classificacao INTEGER,
        Comentarios TEXT,
        FOREIGN KEY (ID_Venda) REFERENCES Vendas (ID_Venda)
    )
    """)

    connection.commit()
    cursor.close()

create_tables(conn)


In [10]:
def insert_data(df, table_name, connection):
    cursor = connection.cursor()
    placeholders = ', '.join(['%s'] * len(df.columns))
    columns = ', '.join(df.columns)
    query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
    for row in df.itertuples(index=False, name=None):
        cursor.execute(query, row)
    connection.commit()
    cursor.close()

# Inserir dados
insert_data(dados_producao, 'Producao', conn)
insert_data(dados_vendas, 'Vendas', conn)
insert_data(dados_devolucao[['ID_Devolucao', 'ID_Venda', 'Quantidade_Devolvida', 'Motivo_Devolucao']], 'Devolucoes', conn)
insert_data(dados_feedback, 'Feedback', conn)

UniqueViolation: duplicate key value violates unique constraint "producao_pkey"
DETAIL:  Key (id_lote)=(1) already exists.


In [11]:
conn.close()